In [115]:
import datetime
import json
import base64
import requests
from bs4 import BeautifulSoup
from dateutil.parser import parse as parse_dt
from google import genai
from google.genai import types
import google.generativeai as genai2
import os
from openai import OpenAI
import time
from pathlib import Path
import numpy as np

DISCOURSE_URL = "https://discourse.onlinedegree.iitm.ac.in"
TDS_CATEGORY = 34

DISCOURSE_TOKEN = "cazThzNHNSSZEi6YGHrPeM82Ff1b108IV%2BMPTOK1ougzuZFIKq%2BScoMCwNy%2BPOPyC4%2BARuR%2BaFV13rNOfTMfRSGiUjgzb3eX4hpD5cAdDwu8zgpee8wm2CElLk%2FUKOQXem4cUlLxh5TPwBWyvxfnvzFjzsyUQJgeoOkgdMtWT4erjf6mSAczBPc0oeBpvsR9iiEFaaG%2B8b9zkznOs%2FHM24FYxRWQAEfCzToIuii%2FJi40RKaqbVVycHejXR%2FzJpHI7NcRVh3gA%2F6jLdSBxbF0caYHYQQ8DymeCHi1xdyURb5UPOI8fU9PL54vSvpDpmkFy1Ydjg%3D%3D--EtEpXQ23oPLlFOy8--0rnvrGpJkIZ73RJBA4OedQ%3D%3D"

session = requests.Session()
session.cookies.set('_t', DISCOURSE_TOKEN, domain="discourse.onlinedegree.iitm.ac.in")

print('')

In [27]:
resp = session.get(f"{DISCOURSE_URL}/c/courses/tds-kb/{TDS_CATEGORY}.json?page=0")
js = resp.json()
print(js['topic_list']['topics'][0]) #check if it worked

{'id': 176077, 'title': 'Project1 - Virtual TA - Discussion Thread [TDS May 2025]', 'fancy_title': 'Project1 - Virtual TA - Discussion Thread [TDS May 2025]', 'slug': 'project1-virtual-ta-discussion-thread-tds-may-2025', 'posts_count': 303, 'reply_count': 199, 'highest_post_number': 312, 'image_url': None, 'created_at': '2025-05-22T21:36:32.840Z', 'last_posted_at': '2025-06-13T11:49:22.285Z', 'bumped': True, 'bumped_at': '2025-06-13T11:49:22.285Z', 'archetype': 'regular', 'unseen': False, 'last_read_post_number': 87, 'unread': 0, 'new_posts': 225, 'unread_posts': 225, 'pinned': True, 'unpinned': None, 'excerpt': 'Please post any questions related to Project 1 \nPlease use markdown code formatting (fenced code blocks beginning with ```) when sharing code (rather than screenshots). It’s easier for us to copy-paste and test. \nDeadlin&hellip;', 'visible': True, 'closed': False, 'archived': False, 'notification_level': 2, 'bookmarked': False, 'liked': False, 'tags': ['term2-2025', 'tds-pro

In [6]:
def parse_dt(dtstr):
    return datetime.datetime.fromisoformat(dtstr.replace("Z", "+00:00"))

START_DATE = datetime.datetime(2025, 1, 1, tzinfo=datetime.timezone.utc)
all_topics = []

for page in range(0, 11):  # pages 0 to 10 inclusive
    url = f"{DISCOURSE_URL}/c/courses/tds-kb/{TDS_CATEGORY}.json?page={page}"
    r = session.get(url)
    data = r.json()
    topics = data.get("topic_list", {}).get("topics", [])
    for t in topics:
        last_posted_at = parse_dt(t["last_posted_at"]) if t.get("last_posted_at") else None
        if last_posted_at and last_posted_at > START_DATE:
            topic_info = {
                "id": t["id"],
                "title": t["title"],
                "slug": t["slug"],
                "created_at": t.get("created_at"),
                "last_posted_at": t.get("last_posted_at"),
                "posts_count": t.get("posts_count"),
                "reply_count": t.get("reply_count"),
                "views": t.get("views"),
                "tags": t.get("tags", []),
                "category_id": t.get("category_id"),
                "last_poster_username": t.get("last_poster_username"),
                "excerpt": t.get("excerpt"),
                "url": f"{DISCOURSE_URL}/t/{t['slug']}/{t['id']}"
            }
            all_topics.append(topic_info)

print(f"Collected {len(all_topics)} topics after {START_DATE.date()}")

# Save to file if needed
import json
with open("tds_topics.json", "w") as f:
    json.dump(all_topics, f, indent=2)


Collected 189 topics after 2025-01-01


## Start Scraping all posts

### Image to Description Function

In [10]:
with open("tds_topics.json", "r") as f:
    all_topics = json.load(f)

In [ ]:
gemini_api_key = os.environ.get('GOOGLE_API_KEY')
client = genai.Client(api_key=gemini_api_key)

In [20]:
def get_image_description(
        image_bytes: bytes,
        model: str = "gemini-2.0-flash",   # or 'gemini-2.0-pro', 'gemini-2.5-pro', …
        max_tokens: int = 300
    ) -> str:
    """Return a concise description of an image."""
    try:
        image_part = types.Part.from_bytes(
            data=image_bytes,
            mime_type="image/jpeg"         # change if PNG/WebP/…
        )

        response = client.models.generate_content(
            model=model,
            contents=[
                ("Describe this image. It is from a data science course forum and likely part of a student's question or instructor response about code, data, or errors. Be concise and clear. Just return the description and keep response concise. Don't cross 500 tokens"),
                image_part
            ],
            config=types.GenerateContentConfig(
                temperature=0.2
            ),
        )
        return response.text.strip()

    except Exception as exc:
        print(f"[Gemini 2.0] {exc}")
        return "Could not generate image description."

In [19]:
from pathlib import Path

def test_image_description():
    test_image_path = "testimage.jpeg"  # Use any small forum screenshot or JPEG image
    if not Path(test_image_path).exists():
        print("Image not found.")
        return

    with open(test_image_path, "rb") as f:
        image_bytes = f.read()

    description = get_image_description(image_bytes)
    print("Description:\n", description)

# Run the test
test_image_description()

Description:
 The image shows a screenshot of a data science course assignment, "TDS 2025-05 Project 1 - Virtual TA". It displays two different due dates: one is "Sat, 31 May, 2025, 11:59 pm IST" and another is "2025-06-14, 23:59 IST". The score is currently 0.


### Process Post Content

In [13]:
def process_post_content(html_content: str, session: requests.Session) -> str:
    """
    Parses HTML, finds images, replaces them with Gemini descriptions,
    and returns clean text.
    """
    soup = BeautifulSoup(html_content, "html.parser")

    # Find all images in the post
    for img in soup.find_all("img"):
        img_url = img.get("src")
        
        # Ensure it's a full URL
        if not img_url.startswith("http"):
            img_url = DISCOURSE_URL + img_url

        try:
            print(f"    -> Found image: {img_url}")
            img_response = session.get(img_url, timeout=10)
            img_response.raise_for_status()
            
            # Get description from Gemini
            description = get_image_description(img_response.content)
            print(f"    -> Description: {description[:80]}...") # Print a snippet
            
            # Replace the <img> tag with a descriptive text node
            description_text = f" [Image: {description}] "
            img.replace_with(description_text)

        except requests.exceptions.RequestException as e:
            print(f"  [!] Failed to download image {img_url}: {e}")
            img.replace_with(" [Image: Could not be downloaded] ")
        except Exception as e:
            print(f"  [!] An unexpected error occurred with image {img_url}: {e}")
            img.replace_with(" [Image: Processing error] ")

    return soup.get_text().strip()


### Start Scraping Posts

In [31]:
START_DATE = datetime.datetime(2025, 1, 1, tzinfo=datetime.timezone.utc)
END_DATE = datetime.datetime(2025, 4, 14, tzinfo=datetime.timezone.utc)

all_posts = []
checkpoint_file = "checkpoint_posts_with_images.json"
start_topic_idx = 69
start_page = 8


# Note: You should have a try/except block here for loading the checkpoint
# For simplicity in this example, we start fresh.

try:
    for topic_idx in range(start_topic_idx, len(all_topics)):
        topic = all_topics[topic_idx]
        topic_id = topic["id"]
        slug = topic["slug"]
        # Use the full URL from your topics data if available
        topic_url = f"{DISCOURSE_URL}/t/{slug}/{topic_id}"
        page = start_page if topic_idx == start_topic_idx else 1

        print(f"Processing topic {topic_idx+1}/{len(all_topics)}: {topic['title']}")

        while True:
            current_url = f"{topic_url}.json?page={page}"
            print(f"  Fetching: {current_url} | Posts so far: {len(all_posts)}")
            resp = session.get(current_url)
            if resp.status_code != 200:
                print(f"    Error {resp.status_code}, skipping page {page} of topic {topic_id}")
                break
            
            topic_data = resp.json()
            posts = topic_data.get("post_stream", {}).get("posts", [])
            if not posts:
                break

            for post in posts:
                created_at = parse_dt(post["created_at"])
                if START_DATE <= created_at <= END_DATE:
                    post_number = post["post_number"]
                    post_url = f"{DISCOURSE_URL}/t/{slug}/{topic_id}/{post_number}"
                    
                    # *** KEY CHANGE HERE ***
                    # Use the new processing function instead of the simple html_to_text
                    print(f"  -> Processing post #{post_number}")
                    content = process_post_content(post["cooked"], session)
                    
                    all_posts.append({
                        "url": post_url,
                        "title": topic.get("title", "No Title"), # Add title for context
                        "text": content
                    })

            if len(posts) < 20:
                break
            page += 1
        
        # Reset page for the next topic
        start_page = 1
        # Consider checkpointing here after each topic

except Exception as e:
    print(f"\nAn exception occurred: {e}. Saving checkpoint.")
    # Add your checkpoint saving logic here
    raise

print(f"\nDone! Total posts collected: {len(all_posts)}")

# Save final output
with open("tds_discourse_posts_processed.json", "w") as f:
    json.dump(all_posts, f, indent=2)


Processing topic 70/189: Urgent Attn Needed: Project 2 Discrepency for GA2_q3 and GA1_q18
  Fetching: https://discourse.onlinedegree.iitm.ac.in/t/urgent-attn-needed-project-2-discrepency-for-ga2-q3-and-ga1-q18/173113.json?page=8 | Posts so far: 0
    Error 404, skipping page 8 of topic 173113
Processing topic 71/189: Project 2 score discrepancy
  Fetching: https://discourse.onlinedegree.iitm.ac.in/t/project-2-score-discrepancy/172832.json?page=1 | Posts so far: 0
Processing topic 72/189: [TDS Jan 2025 official announcement] All score discrepancies
  Fetching: https://discourse.onlinedegree.iitm.ac.in/t/tds-jan-2025-official-announcement-all-score-discrepancies/173025.json?page=1 | Posts so far: 0
Processing topic 73/189: Bonus Marks in TDS for Jan 25
  Fetching: https://discourse.onlinedegree.iitm.ac.in/t/bonus-marks-in-tds-for-jan-25/172246.json?page=1 | Posts so far: 0
  -> Processing post #1
  -> Processing post #2
  -> Processing post #3
  -> Processing post #4
  -> Processing post

## Create document chunks

In [58]:
import json
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Load the data you just scraped
try:
    with open("tds_discourse_posts_processed.json", "r") as f:
        all_posts = json.load(f)
    print(f"Loaded {len(all_posts)} posts.")
except FileNotFoundError:
    print("Error: tds_discourse_posts_processed.json not found. Please run the scraping script first.")
    all_posts = []

# --- Chunking Logic ---

def chunk_data(posts: list) -> list:
    """
    Chunks the text content of posts using a source-aware strategy.
    """
    metadata_store = []
    
    # This splitter will try to split by paragraph, then line, then sentence.
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=100, # A small overlap helps maintain context between chunks
        separators=["\n\n", "\n", ". ", " ", ""]
    )
    
    print("Starting the chunking process...")
    
    for i, post in enumerate(posts):
        if (i + 1) % 100 == 0:
            print(f"  Processing post {i+1}/{len(posts)}")
            
        # Check if the text is substantial enough to be chunked
        if len(post.get('text', '')) > text_splitter._chunk_size:
            # If the text is long, split it into smaller documents
            chunks = text_splitter.split_text(post['text'])
            for chunk in chunks:
                metadata_store.append({
                    "text_chunk": chunk,
                    "url": post['url'],
                    "title": post['title']
                })
        else:
            # If the text is short, treat the whole post as one chunk
            metadata_store.append({
                "text_chunk": post['text'],
                "url": post['url'],
                "title": post['title']
            })
            
    return metadata_store

# Run the chunking process
chunked_metadata = chunk_data(all_posts)

# Save the chunked data
with open("tds_chunked_metadata.json", "w") as f:
    json.dump(chunked_metadata, f, indent=2)

print(f"\nChunking complete. Created {len(chunked_metadata)} chunks.")
print("Saved chunked data to tds_chunked_metadata.json")

Loaded 3025 posts.
Starting the chunking process...
  Processing post 100/3025
  Processing post 200/3025
  Processing post 300/3025
  Processing post 400/3025
  Processing post 500/3025
  Processing post 600/3025
  Processing post 700/3025
  Processing post 800/3025
  Processing post 900/3025
  Processing post 1000/3025
  Processing post 1100/3025
  Processing post 1200/3025
  Processing post 1300/3025
  Processing post 1400/3025
  Processing post 1500/3025
  Processing post 1600/3025
  Processing post 1700/3025
  Processing post 1800/3025
  Processing post 1900/3025
  Processing post 2000/3025
  Processing post 2100/3025
  Processing post 2200/3025
  Processing post 2300/3025
  Processing post 2400/3025
  Processing post 2500/3025
  Processing post 2600/3025
  Processing post 2700/3025
  Processing post 2800/3025
  Processing post 2900/3025
  Processing post 3000/3025

Chunking complete. Created 3678 chunks.
Saved chunked data to tds_chunked_metadata.json


## Generate Embeddings

In [ ]:
client = OpenAI(api_key=os.environ.get('OPENAI_API_KEY'))
OPENAI_EMBEDDING_MODEL = "text-embedding-3-small"

In [66]:
def get_openai_embedding(text: str, model: str = OPENAI_EMBEDDING_MODEL) -> list[float]:
    """
    Generates an embedding for a given text using OpenAI's API.
    Handles retries for transient network errors.
    """
    # OpenAI's API recommends replacing newlines to avoid issues.
    text = text.replace("\n", " ")
    
    for i in range(3): # Retry up to 3 times
        try:
            response = client.embeddings.create(
                input=[text], # API expects a list of texts
                model=model
            )
            # The actual embedding vector is in response.data[0].embedding
            return response.data[0].embedding
        except Exception as e:
            print(f"An error occurred with the OpenAI API: {e}")
            if i < 2:
                print("Retrying in 5 seconds...")
                time.sleep(5)
            else:
                print("Failed to get embedding after multiple retries.")
                return None # Return None on failure


In [96]:

# --- Configuration ---
OPENAI_EMBEDDING_MODEL = "text-embedding-3-small"

# --- Load Data and Generate Embeddings ---
"""
Loads chunked text, generates embeddings via OpenAI, and saves the 
perfectly aligned vector store and metadata.
"""
# try:
#     with open("tds_chunked_metadata.json", "r") as f:
#         original_metadata = json.load(f)
#     print(f"Loaded {len(original_metadata)} text chunks from tds_chunked_metadata.json")
# except FileNotFoundError:
#     print("Error: tds_chunked_metadata.json not found. Please run the chunking script first.")


# --- THIS IS THE KEY CHANGE: Use two new lists to store successful results ---
# final_vectors = []
# final_metadata = []

print(f"Generating embeddings using '{OPENAI_EMBEDDING_MODEL}'...")

for i, item in enumerate(original_metadata):
        
    if (i + 1) % 20 == 0:
        print(f"  Processing chunk {i+1}/{len(original_metadata)}")

    embedding = get_openai_embedding(item['text_chunk'])
    
    # If the embedding was created successfully, add BOTH the vector and metadata to our final lists
    if embedding:
        final_vectors.append(embedding)
        final_metadata.append(item)
    else:
        # If it failed, we print a warning and discard the chunk completely.
        # This keeps the lists in sync.
        print(f"  [!] Failed to create embedding for chunk {i} ('{item['title']}'). Skipping this chunk.")

    time.sleep(0.05)

if len(final_vectors) == 0:
    print("Could not generate any embeddings. Exiting.")

    
print(f"\nSuccessfully generated embeddings for {len(final_vectors)} out of {len(original_metadata)} chunks.")

Generating embeddings using 'text-embedding-3-small'...
  Processing chunk 1600/3678
  Processing chunk 1620/3678
  Processing chunk 1640/3678
  Processing chunk 1660/3678
  Processing chunk 1680/3678
  Processing chunk 1700/3678
  Processing chunk 1720/3678
  Processing chunk 1740/3678
  Processing chunk 1760/3678
  Processing chunk 1780/3678
  Processing chunk 1800/3678
  Processing chunk 1820/3678
  Processing chunk 1840/3678
  Processing chunk 1860/3678
  Processing chunk 1880/3678
  Processing chunk 1900/3678
  Processing chunk 1920/3678
  Processing chunk 1940/3678
  Processing chunk 1960/3678
  Processing chunk 1980/3678
  Processing chunk 2000/3678
  Processing chunk 2020/3678
  Processing chunk 2040/3678
  Processing chunk 2060/3678
  Processing chunk 2080/3678
  Processing chunk 2100/3678
  Processing chunk 2120/3678
  Processing chunk 2140/3678
  Processing chunk 2160/3678
  Processing chunk 2180/3678
  Processing chunk 2200/3678
An error occurred with the OpenAI API: Error 

## Process Course content

In [108]:
import os
import json
from pathlib import Path

def process_course_markdown(folder_path: str) -> list[dict]:
    """
    Scans a directory for markdown files and processes them into a structured list.

    Args:
        folder_path: The path to the folder containing .md files.

    Returns:
        A list of dictionaries, each representing a markdown file.
    """
    # Ensure the folder exists before proceeding
    if not os.path.isdir(folder_path):
        print(f"Error: The directory '{folder_path}' was not found.")
        return []

    print(f"Scanning for markdown files in '{folder_path}'...")
    course_documents = []
    
    # Using pathlib for modern and easy path manipulation
    for file_path in Path(folder_path).rglob("*.md"):
        # The 'stem' is the filename without the final extension
        file_stem = file_path.stem

        # 1. Create the URL as per the specified format
        url = f"https://tds.s-anand.net/#/{file_stem}"

        # 2. Create the title by replacing hyphens and capitalizing
        title = file_stem.replace('-', ' ').title()

        try:
            # 3. Read the text content of the file
            with open(file_path, 'r', encoding='utf-8') as f:
                text = f.read()

            # Append the structured data to our list
            course_documents.append({
                "url": url,
                "title": title,
                "text": text
            })
            print(f"  Processed: {file_path.name}")

        except Exception as e:
            print(f"  [!] Failed to read or process {file_path.name}: {e}")

    return course_documents

# --- Main Execution ---
if __name__ == "__main__":
    # Define the folder where your .md files are located
    COURSE_CONTENT_FOLDER = "jan-course"
    
    # Process the files
    processed_docs = process_course_markdown(COURSE_CONTENT_FOLDER)
    
    if processed_docs:
        # Save the output to a JSON file for later use
        output_filename = "tds_course_content.json"
        with open(output_filename, "w", encoding='utf-8') as f:
            json.dump(processed_docs, f, indent=2, ensure_ascii=False)
        
        print(f"\nProcessing complete.")
        print(f"Successfully processed {len(processed_docs)} markdown files.")
        print(f"Data saved to '{output_filename}'.")
    else:
        print("\nNo markdown files were processed.")

Scanning for markdown files in 'jan-course'...
  Processed: hybrid-rag-typesense.md
  Processed: llm-agents.md
  Processed: live-session-2025-01-30.md
  Processed: development-tools.md
  Processed: convert-pdfs-to-markdown.md
  Processed: marimo.md
  Processed: data-analysis-with-datasette.md
  Processed: hosting-llms-runpod.md
  Processed: live-session-2025-01-20.md
  Processed: visualizing-forecasts-with-excel.md
  Processed: transforming-images.md
  Processed: llm-image-generation.md
  Processed: markdown.md
  Processed: data-storytelling.md
  Processed: regression-with-excel.md
  Processed: uv.md
  Processed: llm-website-scraping.md
  Processed: archive.md
  Processed: data-visualization.md
  Processed: live-sessions.md
  Processed: llm-video-screen-scraping.md
  Processed: wikipedia-data-with-python.md
  Processed: function-calling.md
  Processed: revealjs.md
  Processed: live-session-2025-01-31.md
  Processed: narratives-with-llms.md
  Processed: devtools.md
  Processed: embeddin

In [109]:
# chunking course content
course_content = chunk_data(processed_docs)

Starting the chunking process...
  Processing post 100/144


In [110]:
with open("tds_course_content.json", "w", encoding='utf-8') as f:
            json.dump(course_content, f, indent=2, ensure_ascii=False)

## Create vectors for course content

In [112]:

# --- Configuration ---
OPENAI_EMBEDDING_MODEL = "text-embedding-3-small"

# --- Load Data and Generate Embeddings ---
"""
Loads chunked text, generates embeddings via OpenAI, and saves the 
perfectly aligned vector store and metadata.
"""


print(f"Generating embeddings using '{OPENAI_EMBEDDING_MODEL}'...")

for i, item in enumerate(course_content):
        
    if (i + 1) % 20 == 0:
        print(f"  Processing course chunk {i+1}/{len(course_content)}")

    embedding = get_openai_embedding(item['text_chunk'])
    
    # If the embedding was created successfully, add BOTH the vector and metadata to our final lists
    if embedding:
        final_vectors.append(embedding)
        final_metadata.append(item)
    else:
        # If it failed, we print a warning and discard the chunk completely.
        # This keeps the lists in sync.
        print(f"  [!] Failed to create embedding for chunk {i} ('{item['title']}'). Skipping this chunk.")

    time.sleep(0.05)

if len(final_vectors) == 0:
    print("Could not generate any embeddings. Exiting.")

    
print(f"\nSuccessfully generated embeddings for {len(final_vectors)} out of {len(course_content)} chunks.")

Generating embeddings using 'text-embedding-3-small'...
  Processing course chunk 20/714
  Processing course chunk 40/714
An error occurred with the OpenAI API: Error code: 400 - {'error': {'message': "'$.input' is invalid. Please check the API reference: https://platform.openai.com/docs/api-reference.", 'type': 'invalid_request_error', 'param': None, 'code': None}}
Retrying in 5 seconds...
An error occurred with the OpenAI API: Error code: 400 - {'error': {'message': "'$.input' is invalid. Please check the API reference: https://platform.openai.com/docs/api-reference.", 'type': 'invalid_request_error', 'param': None, 'code': None}}
Retrying in 5 seconds...
An error occurred with the OpenAI API: Error code: 400 - {'error': {'message': "'$.input' is invalid. Please check the API reference: https://platform.openai.com/docs/api-reference.", 'type': 'invalid_request_error', 'param': None, 'code': None}}
Failed to get embedding after multiple retries.
  [!] Failed to create embedding for ch

In [113]:
len(final_vectors) , len(final_metadata)

(4354, 4354)

## Creating numpy compressed file

In [116]:
# Ensure vectors are in a NumPy array (it should be already, but this is good practice)
vectors_np = np.array(final_vectors, dtype=np.float32)

# IMPORTANT: Convert the list of dictionaries to a NumPy array with dtype=object
metadata_np = np.array(final_metadata, dtype=object)

print(f"\nData prepared for saving.")
print(f"Vectors shape: {vectors_np.shape}")
print(f"Metadata shape: {metadata_np.shape}")


# --- Step 3: Save to a single compressed .npz file ---
output_filename = "tds_database.npz"

print(f"\nSaving data to compressed file: {output_filename}...")
np.savez_compressed(
    output_filename,
    vectors=vectors_np,
    metadata=metadata_np
)


Data prepared for saving.
Vectors shape: (4354, 1536)
Metadata shape: (4354,)

Saving data to compressed file: tds_database.npz...


In [121]:
# Load Embedddings
def load_embeddings():
    data = np.load("tds_database.npz", allow_pickle=True)
    data['vectors'], data['metadata']
    
    return data['vectors'], data['metadata']

In [120]:
metadata_np[0]

{'text_chunk': '@carlton I was trying to view a session, but since youtube is very distracting I tried to embed it on a site, but it throws this error\n [Image: The image shows a YouTube video player with an error message: "Video unavailable. Playback on other websites has been disabled by the video owner." A link to "Watch on YouTube" is also displayed.] image2852×1564 131 KB\nWebsite Where I am embedding\nVideo I am trying to watch\nIf possible, please fix this since the youtube as a platform is distracting and has many ads, even though the channel is not monitized.',
 'url': 'https://discourse.onlinedegree.iitm.ac.in/t/about-the-tools-in-data-science-category/23335/40',
 'title': 'About the Tools in Data Science category'}

In [136]:
import base64, io, json, os
from typing import List, Dict, Any, Optional

import numpy as np
from PIL import Image
import pytesseract        # pip install pytesseract pillow
import openai             # pip install openai>=1.0.0

# ---------------------------------------------------------------------------
# External helpers and shared OpenAI client
# ---------------------------------------------------------------------------
try:
    # Prefer the embedding helper (and its client) supplied elsewhere
    from embeddings import get_openai_embedding, client as _shared_client  # type: ignore
except ImportError:
    _shared_client = None  # will fall back to a fresh client if not found

# ---------------------------------------------------------------------------
# Data loading
# ---------------------------------------------------------------------------

def load_embeddings():
    """Load passage vectors and their metadata dicts from *tds_database.npz*."""
    data = np.load("tds_database.npz", allow_pickle=True)
    return data["vectors"], data["metadata"]

# ---------------------------------------------------------------------------
# Utility helpers
# ---------------------------------------------------------------------------

def _image_to_text(b64_img: Optional[str]) -> str:
    """OCR any base‑64‑encoded image; return extracted text or empty string."""
    if not b64_img:
        return ""
    img_bytes = base64.b64decode(b64_img)
    img = Image.open(io.BytesIO(img_bytes))
    return pytesseract.image_to_string(img)


def _get_embedding(text: str, *, model: str = None) -> np.ndarray:
    """Proxy to the user‑supplied *get_openai_embedding* helper and L2‑normalise."""
    vec_list = get_openai_embedding(text)  # type: ignore[arg-type]
    if vec_list is None:
        raise RuntimeError("Failed to obtain embedding for the input text.")
    vec = np.asarray(vec_list, dtype=np.float32)
    return vec / np.linalg.norm(vec)


def _top_k_indices(query_vec: np.ndarray, corpus_vecs: np.ndarray, k: int = 10) -> np.ndarray:
    """Fast cosine‑similarity search using a cached, normalised corpus matrix."""
    if not hasattr(_top_k_indices, "_norm_corpus"):
        _top_k_indices._norm_corpus = (
            corpus_vecs / np.linalg.norm(corpus_vecs, axis=1, keepdims=True)
        )
    sims = _top_k_indices._norm_corpus @ query_vec
    return sims.argsort()[-k:][::-1]

# ---------------------------------------------------------------------------
# Public entry‑point
# ---------------------------------------------------------------------------

def answer_question(
    question: str,
    image_b64: Optional[str] = None,
    openai_api_key: Optional[str] = None,
    k: int = 10,
) -> Dict[str, Any]:
    """Return concise GPT‑4o answer plus *k* supporting passage indices.

    Output schema:
    {
        "answer": "...concise reply...",
        "links": [
            {"index": 42, "text": "Passage title …", "url": "…"},
            ... 10 objects ...
        ]
    }
    """
    # ------------------------------------------------------------------
    # 1️⃣  Combine question + OCR text
    # ------------------------------------------------------------------
    combined_text = question.strip()
    ocr_text = _image_to_text(image_b64)
    if ocr_text:
        combined_text += "\n\nImage text:\n" + ocr_text.strip()

    # ------------------------------------------------------------------
    # 2️⃣  Embed query
    # ------------------------------------------------------------------
    q_vec = _get_embedding(combined_text)

    # ------------------------------------------------------------------
    # 3️⃣  Retrieve passages
    # ------------------------------------------------------------------
    vectors, metadata = load_embeddings()
    top_idxs = _top_k_indices(q_vec, vectors, k=k)

    # ------------------------------------------------------------------
    # 4️⃣  Craft GPT prompt (only text_chunk content)
    # ------------------------------------------------------------------
    context_blocks = [
        f"[{i}] {metadata[i]['title']}: {metadata[i]['text_chunk']}" for i in top_idxs
    ]
    prompt = (
        "You are the TDS Virtual TA. Answer the student's question concisely, "
        "citing the passage indices in brackets when appropriate.\n\n"
        f"Question:\n{question}\n\n"
        "Relevant passages:\n" + "\n".join(context_blocks)
    )

    chat_resp = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.2,
    )
    answer_text = chat_resp.choices[0].message.content.strip()

    # ------------------------------------------------------------------
    # 5️⃣  Build links array with index, title and URL
    # ------------------------------------------------------------------
    links: List[Dict[str, Any]] = [
        {
            "index": int(i),
            "text": metadata[i].get("text_chunk", ""),
            "url": metadata[i].get("url", ""),
        }
        for i in top_idxs
    ]

    return {"answer": answer_text, "links": links}

# ---------------------------------------------------------------------------
# CLI demo --------------------------------------------------------------------
# ---------------------------------------------------------------------------
if __name__ == "__main__":
    demo = answer_question(
        "Should I output the links of course content as well along with posts in Project 1?",
        image_b64=None,
    )
    print(json.dumps(demo, indent=2, ensure_ascii=False))


{
  "answer": "Yes, you should include the links to the course content along with your posts in Project 1. This will help ensure that all relevant information is easily accessible and can enhance the clarity of your submissions. The focus is on the deliverables, but providing context through links is beneficial ([1404]).",
  "links": [
    {
      "index": 1409,
      "text": "by clicking the link\n [Image: The image shows a screenshot of a survey or assignment interface. On the left, \"Project 1\" is selected. On the right, there's a question: \"I have seen Project 1 available at this link and have attempted it.\" with a \"Yes\" radio button option.] image1198×136 9.49 KB\n [Image: The image shows a webpage from a \"Tools in Data Science\" course, specifically the \"Project 1 - LLM-based Automation Agent\" assignment. It includes the project's due date, announcement date, a link to a discussion thread, and a background description of the project's context within a company called DataW

In [127]:
get_openai_embedding("Should I use gpt-4o-mini or gpt-3.5-turbo?")

[-0.018329061567783356,
 -0.02081785351037979,
 0.03349556773900986,
 -0.04510076344013214,
 -0.03104802593588829,
 -0.014382744207978249,
 -0.023430397734045982,
 0.04166320711374283,
 0.04479825869202614,
 -0.0354481004178524,
 0.029810504987835884,
 0.031130528077483177,
 -0.01859031617641449,
 -0.019759085029363632,
 -0.02873798832297325,
 -0.01421774085611105,
 -0.04419324919581413,
 -0.04235071688890457,
 -0.0040803817100822926,
 0.056705962866544724,
 0.04356073960661888,
 0.011921452358365059,
 0.010374550707638264,
 -0.04518326744437218,
 0.03360557183623314,
 0.0077207558788359165,
 -0.00931578315794468,
 -0.012196457013487816,
 0.03872065618634224,
 -0.0007046994287520647,
 0.003595686051994562,
 -0.026097942143678665,
 -0.026084192097187042,
 -0.048840828239917755,
 -0.04776830971240997,
 -0.017985304817557335,
 0.05106836557388306,
 0.06523110717535019,
 0.0032398987095803022,
 0.011618946678936481,
 0.033633071929216385,
 -0.02795422449707985,
 0.02495667338371277,
 0.031